### Schema Evolution

**MERGE SCHEMA**

In [0]:
my_data = [(1, 'food', 10),(2, 'drink', 20),(3, 'food', 30),(4, 'drink', 40)]
my_schema = "id INT, category STRING, amt INT"

df = spark.createDataFrame(my_data, schema=my_schema)

In [0]:
df_new = df.union(spark.createDataFrame([(5, 'food', 50)], schema=my_schema))

In [0]:
from pyspark.sql.functions import *

In [0]:
df_new = df_new.withColumn("flag", lit(1))

In [0]:
df_new.display()

In [0]:
df_new.write.format("delta")\
    .mode("append")\
        .option("path", "abfss://raw@storagedeltalakerolapati.dfs.core.windows.net/sch_tbl")\
            .option("mergeSchema", "true")\
            .save()

In [0]:
df = spark.read.format("delta").load('abfss://raw@storagedeltalakerolapati.dfs.core.windows.net/sch_tbl')
df.display()

### EXPLICIT SCHEMA UPDATES

**ADD A COLUMN**

In [0]:
%sql
alter table delta_catalog.raw.ext_table_dml
add column flag STRING;

In [0]:
%sql
select * from delta_catalog.raw.ext_table_dml

In [0]:
%sql
alter table delta_catalog.raw.ext_table_dml
add column newcol STRING after id;

In [0]:
%sql
select * from delta_catalog.raw.ext_table_dml

**Reordering Columns**

In [0]:
%sql
alter table delta_catalog.raw.ext_table_dml
alter column newcol after flag

In [0]:
%sql
select * from delta_catalog.raw.ext_table_dml

**Rename Columns**

In [0]:
%sql
ALTER TABLE delta_catalog.raw.ext_table_dml 
SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5'
);

In [0]:
%sql
alter table delta_catalog.raw.ext_table_dml
rename column newcol to newflag;

In [0]:
%sql
ALTER TABLE delta_catalog.raw.ext_table_dml 
SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5'
);

In [0]:
%sql
reorg table delta_catalog.raw.ext_table_dml apply (purge);